# notebook to merge ADNI spreadsheets together 

In [17]:
import pandas as pd
from dateutil.parser import parse
from copy import deepcopy

In [522]:
path_csv = '/Users/AngelaTam/Desktop/adsf/adni_csv/'

In [523]:
adni_merge = pd.read_csv(path_csv + 'ADNIMERGE_20171206.csv')
upenn_csf = pd.read_csv(path_csv + 'UPENNBIOMK_MASTER.csv')
ucb_av45 = pd.read_csv(path_csv + 'UCBERKELEYAV45_11_14_17.csv')
mem_ef = pd.read_csv(path_csv + 'UWNPSYCHSUM_10_27_17.csv')
neurobat = pd.read_csv(path_csv + 'NEUROBAT.csv')

/Users/AngelaTam/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## filter out upenn csf data 

In [524]:
# only keep the correct batch for CSF based on ADNI reccomendations
for i,row in upenn_csf.iterrows():
    batch = row[2]
    if batch == 'MEDIAN':
        upenn_csf.drop(i,axis=0,inplace=True)
    if batch == 'UPENNBIOMK2':
        upenn_csf.drop(i,axis=0,inplace=True)
    if batch == 'UPENNBIOMK4':
        upenn_csf.drop(i,axis=0,inplace=True)

In [525]:
# only keep the one measurement per visit for each subject
upenn_csf.drop_duplicates(subset=['VISCODE', 'DRWDTE'], keep='first',inplace=True)

## filter out the uc berkeley av45 data 

In [526]:
av45 = ucb_av45.filter(['RID','VISCODE2','SUMMARYSUVR_WHOLECEREBNORM_1.11CUTOFF'], axis=1)

In [527]:
av45.rename(columns={'VISCODE2':'VISCODE'}, inplace=True)

## filter out the uwn psych data 

In [528]:
mem_ef = mem_ef.filter(['RID','VISCODE2','ADNI_MEM','ADNI_EF'], axis=1)

In [529]:
mem_ef.rename(columns={'VISCODE2':'VISCODE'}, inplace=True)

## filter out the neuropsych battery data 

In [530]:
neurobat = neurobat.filter(['RID', 'VISCODE2', 'CLOCKSCOR', 'DSPANFOR', 'DSPANBAC', 'BNTTOTAL'], axis=1)

In [531]:
neurobat.rename(columns={'VISCODE2':'VISCODE'}, inplace=True)

## merge the csvs together to make one big csv with longitudinal data

In [532]:
merged_df = pd.merge(adni_merge, mem_ef, how='left', on=['RID','VISCODE'])

In [533]:
merged_df = pd.merge(merged_df, neurobat, how='left', on=['RID', 'VISCODE'])

In [534]:
merged_df = pd.merge(merged_df, upenn_csf, how='left', on=['RID','VISCODE'])

In [535]:
merged_df = pd.merge(merged_df, av45, how='left', on=['RID','VISCODE'])

## determine who is a converter

In [536]:
# sort the df by RID and Years_bl
merged_df.sort_values(by=['RID','Years_bl'], ascending=True, inplace=True)

In [537]:
merged_df.reset_index(drop=True, inplace=True)

In [538]:
# forward fill
missing_dx = [x for x in merged_df.RID.unique() if any([not pd.notnull(y) for y in merged_df[merged_df.RID==x]['DX']])]
for i,sub in enumerate(missing_dx):
    tmp = merged_df[merged_df.RID==sub]['DX'].fillna(method='ffill')
    merged_df.loc[merged_df.RID==sub,'DX'] = tmp.values

In [539]:
# get the final DX for each subject and put into new column
dx_dict = {}
for sid in merged_df.RID.unique():
    final_dx = merged_df[merged_df.RID == sid]['DX'].values[-1]
    dx_dict.update({sid: final_dx})
for i,row in merged_df.iterrows():
    sid = row['RID']
    merged_df.ix[i, 'final_DX'] = dx_dict[sid]

/Users/AngelaTam/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:8: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


In [540]:
merged_df['final_DX'].unique()

array(['MCI', 'Dementia', 'CN', nan], dtype=object)

In [541]:
merged_df['DX'].unique()

array(['CN', 'MCI', 'Dementia', nan], dtype=object)

In [542]:
conv_dict = {}
for sid in merged_df.RID.unique():
    # compare baseline and final DX to determine conversion
    bl_dx = merged_df[merged_df.RID == sid]['DX_bl'].values[0]
    final_dx = merged_df[merged_df.RID == sid]['DX'].values[-1]
    if pd.isnull(final_dx):
        status = 'NaN'
    if bl_dx == 'CN' and final_dx == 'CN':
        status = 'stable'
    if bl_dx == 'LMCI' and final_dx == 'MCI':
        status = 'stable'
    if bl_dx == 'EMCI' and final_dx == 'MCI':
        status = 'stable'
    if bl_dx == 'AD' and final_dx == 'Dementia':
        status = 'stable'
    if bl_dx == 'CN' and final_dx == 'MCI':
        status = 'CN to MCI'
    if bl_dx == 'CN' and final_dx == 'Dementia':
        status = 'CN to Dementia'
    if bl_dx == 'LMCI' and final_dx == 'CN':
        status = 'MCI to CN'
    if bl_dx == 'LMCI' and final_dx == 'Dementia':
        status = 'MCI to Dementia'
    if bl_dx == 'EMCI' and final_dx == 'CN':
        status = 'MCI to CN'
    if bl_dx == 'EMCI' and final_dx == 'Dementia':
        status = 'MCI to Dementia'
    if bl_dx == 'AD' and final_dx == 'MCI':
        status = 'Dementia to MCI'
    if bl_dx == 'AD' and final_dx == 'CN':
        status = 'Dementia to CN'
    conv_dict.update({sid: status})
for i,row in merged_df.iterrows():
    sid = row['RID']
    merged_df.ix[i, 'conversion'] = conv_dict[sid]

In [543]:
# create dummy variables for conversion
for i,row in merged_df.iterrows():
    status = row[merged_df.columns.get_loc('conversion')]
    if status == 'stable':
        merged_df.ix[i,'conv_2_ad'] = 0
        merged_df.ix[i,'conv_2_mci'] = 0
    if status == 'CN to MCI':
        merged_df.ix[i,'conv_2_ad'] = 0
        merged_df.ix[i,'conv_2_mci'] = 1
    if status == 'MCI to Dementia':
        merged_df.ix[i,'conv_2_ad'] = 1
        merged_df.ix[i,'conv_2_mci'] = 0
    if status == 'MCI to CN':
        merged_df.ix[i,'conv_2_ad'] = 0
        merged_df.ix[i,'conv_2_mci'] = 0
    if status == 'CN to Dementia':
        merged_df.ix[i,'conv_2_ad'] = 1
        merged_df.ix[i,'conv_2_mci'] = 0
    if status == 'Dementia to CN':
        merged_df.ix[i,'conv_2_ad'] = 0
        merged_df.ix[i,'conv_2_mci'] = 0

## create dummy variables for dx, gender, apoe4 

In [544]:
# create dummy variables for diagnosis
for i,row in merged_df.iterrows():
    dx = row[merged_df.columns.get_loc('DX')]
    if dx == 'CN':
        merged_df.ix[i,'CN'] = 1
        merged_df.ix[i,'MCI'] = 0
        merged_df.ix[i,'AD'] = 0
    elif dx == 'MCI':
        merged_df.ix[i,'CN'] = 0
        merged_df.ix[i,'MCI'] = 1
        merged_df.ix[i,'AD'] = 0
    elif dx == 'Dementia':
        merged_df.ix[i,'CN'] = 0
        merged_df.ix[i,'MCI'] = 0
        merged_df.ix[i,'AD'] = 1
    else:
        merged_df.ix[i,'CN'] = 'NaN'
        merged_df.ix[i,'MCI'] = 'NaN'
        merged_df.ix[i,'AD'] = 'NaN'

In [545]:
# create dummy variable for gender (0 = female, 1 = male)
sex_dict = {}
for sid in merged_df.RID.unique():
    sex = merged_df[merged_df.RID == sid]['PTGENDER'].values[0]
    if sex == 'Male':
        gender = 1
    elif sex == 'Female':
        gender = 0
    else:
        gender = 'NaN'
    sex_dict.update({sid: gender})
for i,row in merged_df.iterrows():
    sid = row['RID']
    merged_df.ix[i, 'gender'] = sex_dict[sid]

In [546]:
# create binary variable for APOE4 carriership
e4_dict = {}
for sid in merged_df.RID.unique():
    e4_no = merged_df[merged_df.RID == sid]['APOE4'].values[0]
    if e4_no == 0:
        e4_bin = 0
    elif e4_no > 0:
        e4_bin = 1
    else:
        e4_bin = 'NaN'
    e4_dict.update({sid: e4_bin})
for i,row in merged_df.iterrows():
    sid = row['RID']
    merged_df.ix[i,'APOE4_bin'] = e4_dict[sid]

In [547]:
merged_df.to_csv(path_csv + 'adnimerge_upenn_unw_av45_neurobat.csv',index=False)

## ADNI1 baseline

In [548]:
adni1_mri = pd.read_csv('/Users/AngelaTam/Desktop/adsf/adni1_bl_vbm_qc/adni1_screening_metadata.csv')

In [549]:
adni1_merged = deepcopy(merged_df)

In [550]:
# grab just ADNI1 from merged_df
for i,row in adni1_merged.iterrows():
    colprot = row[adni1_merged.columns.get_loc('COLPROT')]
    if colprot != 'ADNI1':
        adni1_merged.drop(i, axis=0, inplace=True)

In [551]:
# grab the baselines from merged_df
for i,row in adni1_merged.iterrows():
    viscode = row[adni1_merged.columns.get_loc('VISCODE')]
    if viscode != 'bl':
        adni1_merged.drop(i, axis=0, inplace=True)

In [552]:
# filter the mri data
adni1_mr = adni1_mri.filter(['RID','Age','Acq Date','vbm_qc'])

In [553]:
adni1_mr.rename(columns={'Age':'age_scan', 'Acq Date': 'scan_date'}, inplace=True)

In [554]:
# merge the MRI data with rest of data
adni1 = pd.merge(adni1_mr, adni1_merged, how='left', on='RID')

In [555]:
adni1.to_csv('/Users/AngelaTam/Desktop/adsf/adni1_bl_vbm_qc/adni1_bl_demog_qc.csv',index=False)

## ADNI2 baseline

In [556]:
adni2_scan = pd.read_csv('/Users/AngelaTam/Desktop/adsf/adni2_bl_vbm_qc/adni2_baseline_mprage_3T_12_05_2017.csv')
adni2_qc = pd.read_csv('/Users/AngelaTam/Desktop/adsf/adni2_bl_vbm_qc/adni2_bl_vbm_dartel_20171201_qc.csv',
                      skipinitialspace=True)

In [557]:
adni2_merged = deepcopy(merged_df)

In [558]:
# grab just ADNI2 from merged_df
for i,row in adni2_merged.iterrows():
    colprot = row[adni2_merged.columns.get_loc('COLPROT')]
    if colprot != 'ADNI2':
        adni2_merged.drop(i, axis=0, inplace=True)

In [559]:
# get the baselines by taking first instance of each subject
adni2_merged.drop_duplicates(subset='RID',keep='first',inplace=True)

In [560]:
adni2_qc.rename(columns={'rid':'RID'},inplace=True)

In [561]:
for i,row in adni2_scan.iterrows():
    sid = row[adni2_scan.columns.get_loc('Subject')]
    rid = sid[6:]
    adni2_scan.ix[i,'RID'] = int(rid)

/Users/AngelaTam/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:4: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


In [562]:
# merge the MRI data together
adni2_mri = pd.merge(adni2_qc, adni2_scan, how='left', on='RID')

In [563]:
print(adni2_mri.columns)

Index(['subject', 'RID', 'vbm_qc', 'vbm_comments', 'scan_date', 'scan',
       'Image Data ID', 'Subject', 'Group', 'Sex', 'Age', 'Visit', 'Modality',
       'Description', 'Type', 'Acq Date', 'Format', 'Downloaded'],
      dtype='object')


In [564]:
adni2_mri = adni2_mri.filter(['RID','Age','scan_date','vbm_qc'], axis=1)

In [565]:
adni2_mri.rename(columns={'Age':'age_scan'}, inplace=True)

In [566]:
adni2 = pd.merge(adni2_mri, adni2_merged, how='left', on='RID')

In [567]:
adni2.to_csv('/Users/AngelaTam/Desktop/adsf/adni2_bl_vbm_qc/adni2_bl_demog_qc.csv',index=False)

## put ADNI1 and ADNI2 baselines together for subtyping pipeline

In [568]:
# get rid of subjects that rolled over from ADNI1 in ADNI2 baseline df
adni2.drop(adni2[adni2.RID < 2000].index, inplace=True)

In [569]:
# set index to RID
adni2.set_index('RID',inplace=True)
adni1.set_index('RID',inplace=True)

In [570]:
frames = [adni1, adni2]
all_df = pd.concat(frames)

In [571]:
all_df.reset_index(inplace=True)

In [572]:
all_df.to_csv('/Users/AngelaTam/Desktop/adsf/adni1_adni2_bl_demog_qc.csv', index=False)

### get the gmv and tiv for each dataset

In [573]:
adni1_gmv = pd.read_csv('/Users/AngelaTam/Desktop/adsf/adni1_bl_vbm_qc/adni1_bl_vbm_mean_gm_tiv.csv',
                       skipinitialspace=True)
adni2_gmv = pd.read_csv('/Users/AngelaTam/Desktop/adsf/adni2_bl_vbm_qc/adni2_bl_vbm_mean_gm_tiv.csv',
                       skipinitialspace=True)

In [574]:
# get rid of whitespaces in column names
adni1_gmv.rename(columns=lambda x: x.strip(),inplace=True)
# rename ' ' to subject
adni1_gmv.rename(index=str,columns={'':'subject'},inplace=True)
# get rid of trailing spaces in subject names in wt_df
for i,row in adni1_gmv.iterrows():
    subj = row[adni1_gmv.columns.get_loc("subject")]
    subj = subj.rstrip()
    adni1_gmv.ix[i,'subject'] = subj

/Users/AngelaTam/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:9: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


In [575]:
# get rid of whitespaces in column names
adni2_gmv.rename(columns=lambda x: x.strip(),inplace=True)
# rename ' ' to subject
adni2_gmv.rename(index=str,columns={'':'subject'},inplace=True)
# get rid of trailing spaces in subject names in wt_df
for i,row in adni2_gmv.iterrows():
    subj = row[adni2_gmv.columns.get_loc("subject")]
    subj = subj.rstrip()
    adni2_gmv.ix[i,'subject'] = subj

/Users/AngelaTam/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:9: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


In [576]:
# get rid of subjects that rolled over from ADNI1 in ADNI2 baseline df
adni2_gmv.drop(adni2_gmv[adni2_gmv.RID < 2000].index, inplace=True)

In [577]:
adni1_gmv.set_index('subject', inplace=True)
adni2_gmv.set_index('subject', inplace=True)

In [578]:
frames = [adni1_gmv, adni2_gmv]
gmv_df = pd.concat(frames)

In [579]:
gmv_df.reset_index(inplace=True)

In [580]:
model_df = pd.merge(all_df, gmv_df, how='left', on='RID')

In [581]:
cols = list(model_df)
cols.insert(0, cols.pop(cols.index('subject')))
model_df = model_df.ix[:,cols]

/Users/AngelaTam/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:3: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  app.launch_new_instance()


In [582]:
model_df.to_csv('/Users/AngelaTam/Desktop/adsf/adni1_adni2_bl_vbm_model_niak.csv',index=False,na_rep='NaN')
model_df.to_csv('/Users/AngelaTam/Desktop/adsf/adni1_adni2_bl_vbm_model_python.csv',index=False)